# CSC325  Milestone 2  
**Authors:** Reeya Patel & Megan Mohr  
**Dataset:** USA House Sales Dataset  
**Platform:** AWS RDS (MySQL)  

### Description
In this milestone, we connected our MySQL database hosted on AWS RDS and implemented the normalized schema we designed in Milestone 1.  
We then used Python and PyMySQL to parse and bulk-load data from multiple CSV files (`cities.csv`, `agents.csv`, `statuses.csv`, `properties.csv`, and `listings.csv`) into the database.  

Each CSV was cleaned, validated, and inserted into the corresponding table using automated scripts with parameterized SQL queries.  
All foreign key relationships were verified by joining tables to confirm that properties, agents, cities, and statuses are correctly linked.  

### Verification
At the end of the notebook, we included verification queries to display:
- The total row counts for each table  
- Sample joined data showing each property’s city, agent, and status  

This confirms that the data model, schema, and AWS RDS connection were successfully implemented for Milestone 2.

In [1]:
!pip install pymysql python-dotenv pandas

In [2]:
# aws connection set up 
import pymysql

HOST = "database-1.cfqmw8c6u8ay.us-east-2.rds.amazonaws.com"
PORT = 3306
USER = "admin"
PASSWORD = "S0qRV3&dL},E{=DT"
DB_NAME = "house_sales"

def getconn(db=DB_NAME):
    return pymysql.connect(
        host=HOST, port=PORT, user=USER, password=PASSWORD,
        database=db, autocommit=True
    )

with getconn() as conn, conn.cursor() as cur:
    cur.execute("SELECT DATABASE()")
    print("Connected. Current DB:", cur.fetchone()[0])
    cur.execute("SHOW TABLES")
    print("Tables:", [r[0] for r in cur.fetchall()])


Connected. Current DB: house_sales
Tables: ['Agent', 'City', 'Listing', 'Property', 'Status']


In [4]:
# load city table
import pandas as pd

df = pd.read_csv("cities.csv")

def clean(s):
    if pd.isna(s): return None
    s = str(s).strip()
    return s if s else None

rows = [(clean(r.city), clean(r.state), clean(r.zipcode)) for r in df.itertuples(index=False)]
sql  = "INSERT IGNORE INTO City (city, state, zipcode) VALUES (%s,%s,%s)"

with getconn() as conn, conn.cursor() as cur:
    cur.executemany(sql, rows)
    print(f"Inserted {cur.rowcount} new City rows.")
    cur.execute("SELECT COUNT(*) FROM City")
    print("City row count:", cur.fetchone()[0])

Inserted 0 new City rows.
City row count: 2992


In [5]:
#load agent table 
import pandas as pd

df = pd.read_csv("agents.csv")

def clean(s):
    if pd.isna(s): return None
    s = str(s).strip()
    return s if s else None

rows = [(clean(r.listing_agent),) for r in df.itertuples(index=False)]
sql  = "INSERT IGNORE INTO Agent (listing_agent) VALUES (%s)"

with getconn() as conn, conn.cursor() as cur:
    cur.executemany(sql, rows)
    print(f"Inserted {cur.rowcount} new Agent rows.")
    cur.execute("SELECT COUNT(*) FROM Agent")
    print("Agent row count:", cur.fetchone()[0])

Inserted 0 new Agent rows.
Agent row count: 5


In [6]:
# load status table 
import pandas as pd

df = pd.read_csv("statuses.csv")

def clean(s):
    if pd.isna(s): return None
    s = str(s).strip()
    return s if s else None

rows = [(clean(r.status),) for r in df.itertuples(index=False)]
sql  = "INSERT IGNORE INTO Status (status) VALUES (%s)"

with getconn() as conn, conn.cursor() as cur:
    cur.executemany(sql, rows)
    print(f"Inserted {cur.rowcount} new Status rows.")
    cur.execute("SELECT COUNT(*) FROM Status")
    print("Status row count:", cur.fetchone()[0])

Inserted 0 new Status rows.
Status row count: 3


In [7]:
#load properties table 
import pandas as pd

props = pd.read_csv("properties.csv")

def clean_str(x):
    if pd.isna(x): return None
    s = str(x).strip()
    return s if s else None

def clean_int(x):
    try:
        if pd.isna(x) or str(x).strip()=="":
            return None
        return int(float(x))
    except:
        return None

def clean_dec(x):
    try:
        if pd.isna(x) or str(x).strip()=="":
            return None
        return float(x)
    except:
        return None

# City lookup
city_lookup = {}
with getconn() as conn, conn.cursor() as cur:
    cur.execute("SELECT city_id, city, state, zipcode FROM City")
    for city_id, c, s, z in cur.fetchall():
        key = ((c or "").strip().lower(), (s or "").strip().upper(), (z or "").strip())
        city_lookup[key] = city_id

def lookup_city_id(city, state, zipcode):
    key = ((str(city or "")).strip().lower(),
           (str(state or "")).strip().upper(),
           (str(zipcode or "")).strip())
    return city_lookup.get(key)  # may be None; FK allows NULL

# Keep only needed columns
props = props.rename(columns={
    "address":"address","city":"city","state":"state","zipcode":"zipcode",
    "price":"price","bedrooms":"bedrooms","bathrooms":"bathrooms",
    "area_sqft":"area_sqft","lot_size":"lot_size","year_built":"year_built",
    "days_on_market":"days_on_market","property_type":"property_type"
})
cols = ["address","city","state","zipcode","price","bedrooms","bathrooms",
        "area_sqft","lot_size","year_built","days_on_market","property_type"]
props = props[cols]

rows = []
for r in props.itertuples(index=False):
    city_id = lookup_city_id(r.city, r.state, r.zipcode)
    rows.append((
        clean_str(r.address), city_id, clean_dec(r.price), clean_int(r.bedrooms),
        clean_dec(r.bathrooms), clean_int(r.area_sqft), clean_dec(r.lot_size),
        clean_int(r.year_built), clean_int(r.days_on_market), clean_str(r.property_type)
    ))

sql = """
INSERT IGNORE INTO Property
(address, city_id, price, bedrooms, bathrooms, area_sqft, lot_size, year_built, days_on_market, property_type)
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
"""

with getconn() as conn, conn.cursor() as cur:
    cur.executemany(sql, rows)
    print(f"Inserted {cur.rowcount} new rows into Property.")
    cur.execute("SELECT COUNT(*) FROM Property")
    print("Property row count:", cur.fetchone()[0])


Inserted 0 new rows into Property.
Property row count: 3000


In [8]:
# check 
import pandas as pd

df_list = pd.read_csv("listings.csv")
print("Listings sample:")
display(df_list.head())

with getconn() as conn, conn.cursor() as cur:
    cur.execute("SELECT property_id FROM Property"); props_ids = {r[0] for r in cur.fetchall()}
    cur.execute("SELECT agent_id    FROM Agent");     agent_ids = {r[0] for r in cur.fetchall()}
    cur.execute("SELECT status_id   FROM Status");    status_ids= {r[0] for r in cur.fetchall()}

missing_props  = (~df_list['property_id'].isin(props_ids)).sum()
missing_agents = (~df_list['agent_id'   ].isin(agent_ids)).sum()
missing_status = (~df_list['status_id'  ].isin(status_ids)).sum()
print("Missing property_id rows:", missing_props)
print("Missing agent_id rows   :", missing_agents)
print("Missing status_id rows  :", missing_status)

Listings sample:


,property_id,agent_id,status_id,listing_url
0,1,1,1,https://www.zillow.com/homedetails/80374762_zpid/
1,2,2,2,https://www.zillow.com/homedetails/86143665_zpid/
2,3,3,1,https://www.zillow.com/homedetails/37082403_zpid/
3,4,4,3,https://www.zillow.com/homedetails/39318132_zpid/
4,5,4,3,https://www.zillow.com/homedetails/22454634_zpid/


Missing property_id rows: 0
Missing agent_id rows   : 0
Missing status_id rows  : 0


In [9]:
# load listings 
def clean_str(x):
    if pd.isna(x): return None
    s = str(x).strip()
    return s if s else None

def clean_int(x):
    try:
        return int(x)
    except:
        return None

rows = [
    (clean_int(r.property_id), clean_int(r.agent_id), clean_int(r.status_id), clean_str(r.listing_url))
    for r in df_list.itertuples(index=False)
]

sql = """
INSERT IGNORE INTO Listing
(property_id, agent_id, status_id, listing_url)
VALUES (%s,%s,%s,%s)
"""

with getconn() as conn, conn.cursor() as cur:
    cur.executemany(sql, rows)
    print(f"Inserted {cur.rowcount} new rows into Listing.")
    cur.execute("SELECT COUNT(*) FROM Listing")
    print("Listing row count:", cur.fetchone()[0])

# quick human-readable sample
with getconn() as conn, conn.cursor() as cur:
    cur.execute("""
        SELECT l.listing_id, p.address, c.city, a.listing_agent, s.status, l.listing_url
        FROM Listing l
        JOIN Property p ON l.property_id = p.property_id
        JOIN City c      ON p.city_id = c.city_id
        JOIN Agent a     ON l.agent_id = a.agent_id
        JOIN Status s    ON l.status_id = s.status_id
        ORDER BY l.listing_id
        LIMIT 5
    """)
    for row in cur.fetchall():
        print(row)

Inserted 3000 new rows into Listing.
Listing row count: 3000
(1, '5926 Oak Ave, San Diego, CA 65383', 'San Diego', 'Alex Johnson - Compass', 'For Sale', 'https://www.zillow.com/homedetails/80374762_zpid/')
(2, '9583 Oak Ave, Fresno, IL 79339', 'Fresno', 'Emily Davis - Century 21', 'Sold', 'https://www.zillow.com/homedetails/86143665_zpid/')
(3, '8224 Oak Ave, Sacramento, TX 87393', 'Sacramento', 'Mike Lee - Coldwell Banker', 'For Sale', 'https://www.zillow.com/homedetails/37082403_zpid/')
(4, '232 Oak Ave, Fresno, TX 38666', 'Fresno', 'John Doe - RE/MAX', 'Pending', 'https://www.zillow.com/homedetails/39318132_zpid/')
(5, '5446 Pine Rd, Los Angeles, CA 23989', 'Los Angeles', 'John Doe - RE/MAX', 'Pending', 'https://www.zillow.com/homedetails/22454634_zpid/')


In [10]:
# final 
with getconn() as conn, conn.cursor() as cur:
    print("Table Counts:")
    for tbl in ["City","Agent","Status","Property","Listing"]:
        cur.execute(f"SELECT COUNT(*) FROM {tbl}")
        print(f"{tbl:10s}: {cur.fetchone()[0]} rows")

    print("\nSample Joined Data:")
    cur.execute("""
        SELECT l.listing_id, p.address, c.city, a.listing_agent, s.status
        FROM Listing l
        JOIN Property p ON l.property_id = p.property_id
        JOIN City c      ON p.city_id = c.city_id
        JOIN Agent a     ON l.agent_id = a.agent_id
        JOIN Status s    ON l.status_id = s.status_id
        LIMIT 5
    """)
    for row in cur.fetchall():
        print(row)

Table Counts:
City      : 2992 rows
Agent     : 5 rows
Status    : 3 rows
Property  : 3000 rows
Listing   : 3000 rows

Sample Joined Data:
(69, '8789 Oak Ave, San Francisco, TX 99388', 'San Francisco', 'Alex Johnson - Compass', 'Sold')
(72, '841 Maple Dr, Fresno, FL 78532', 'Fresno', 'Alex Johnson - Compass', 'Sold')
(79, '3106 Oak Ave, Fresno, IL 66989', 'Fresno', 'Alex Johnson - Compass', 'Sold')
(85, '6181 Main St, Fresno, NY 10445', 'Fresno', 'Alex Johnson - Compass', 'Sold')
(104, '4328 Pine Rd, San Diego, TX 55033', 'San Diego', 'Alex Johnson - Compass', 'Sold')
